In [9]:
import numpy as np
import pandas as pd

song_df = pd.read_csv('./data/raw/spotify_data.csv')                                    # 음악 정보가 담긴 df
movie_df = pd.read_csv('./data/processed/movies_processed_final.csv')                   # 영화의 ost 곡 리스트가 있는 df

# print(song_df.info())
print(song_df.describe(include='object'))

        artist_name track_name                track_id        genre
count       1159749    1159763                 1159764      1159764
unique        64158     882360                 1159764           82
top     Traditional      Intro  53QF56cjZA9RTuuMZDrSA6  black-metal
freq           4058        466                       1        21852


In [10]:
movie_df["track_title"] = movie_df["track_list"].astype(str).str.split("@")             # @을 기점으로 나누고
movie_ost = movie_df["track_title"].explode().str.lower().str.strip().dropna().unique() # movie_ost라는 레퍼런스 df를 만든다.
                                                                                            # track 타이틀을 모두 소문자로 만들고 공백을 제거한다.

song_df = song_df.dropna(subset=["track_name"])                                         # song_df의 공백 데이터가 있으면 지운다.   
song_df["title_key"] = song_df["track_name"].str.lower().str.strip()                    # 타이틀을 모두 소문자화 + 공백 제거
song_df = song_df.drop_duplicates(subset=["title_key"], keep=False)                     # keep=False로 해서 제목 중복되는 곡들은 모두 제거

ost_song_df = song_df[song_df["title_key"].isin(movie_ost)].copy()

In [11]:
# 병합과정

movie_df2 = movie_df.explode("track_title")

movie_df2["title_key"] = movie_df2["track_title"].astype(str).str.lower().str.strip()

movie_df2 = movie_df2.drop(columns=["track_list"])
ost_song_df = ost_song_df.drop(columns=["track_id","track_name"])

merged_df = pd.merge(
    movie_df2,
    ost_song_df,
    on="title_key",
    how="inner"
)

merged_df = merged_df.drop(columns=["title_key","Unnamed: 0_x", "Unnamed: 0_y"]).reset_index(drop=True)

merged_df.to_csv('./ost_list_endgame.csv', encoding='UTF-8')

In [12]:
merged_df["track_title"].value_counts()

track_title
 Bridal Chorus                      98
 The Star Spangled Banner           67
 Taps                               57
 Hail to the Chief                  43
 Sobre las olas (Over the Waves)    39
                                    ..
 I'm Gonna Make You My Wife          1
 This Is How It Works                1
 Wey U                               1
Exhale (Shoop Shoop)                 1
 I Hear Bells (Wedding Bells)        1
Name: count, Length: 19169, dtype: int64

In [ ]:
pd.set_option('display.max_columns',None)
merged_df[merged_df['track_title']=='Bridal Chorus']

,imdb_id,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,track_title,artist_name,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
892,tt0042451,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
7879,tt0054460,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
7908,tt0061955,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
9511,tt0466342,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
10438,tt0023960,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
11694,tt1084950,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
23234,tt0052135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4
23652,tt2415112,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bridal Chorus,Richard Wagner,7,2004,opera,0.298,0.195,10,-18.61,1,0.0367,0.994,0.175,0.511,0.226,77.459,38840,4


: 